# Writing an Algorithm that Uses SMA 20, 50, 100 Trading

## Data Loading

In [7]:
# Import the required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

In [10]:
# Read the msft.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
msft_df = pd.read_csv(
    Path("../data/MSFTdata.csv"),
    index_col="timestamp",
    parse_dates=True,
    infer_datetime_format=True
)

In [11]:
# Review the DataFrame
msft_df.head()

,open,high,low,close,volume,symbol
timestamp,,,,,,
2017-06-01 04:00:00+00:00,70.24,70.61,69.451,70.10,21922908,MSFT
2017-06-02 04:00:00+00:00,70.44,71.86,70.240,71.76,35470261,MSFT
2017-06-05 04:00:00+00:00,71.97,72.89,71.810,72.28,33317996,MSFT
2017-06-06 04:00:00+00:00,72.30,72.62,72.270,72.52,31511065,MSFT
2017-06-07 04:00:00+00:00,72.67,72.77,71.950,72.39,22302769,MSFT


In [12]:

# Filter the date index and close columns
signals_df = msft_df.loc[:,["close"]]

# Review the DataFrame
signals_df.head()

,close
timestamp,
2017-06-01 04:00:00+00:00,70.10
2017-06-02 04:00:00+00:00,71.76
2017-06-05 04:00:00+00:00,72.28
2017-06-06 04:00:00+00:00,72.52
2017-06-07 04:00:00+00:00,72.39


In [13]:
# Use hvplot to visualize the data
signals_df.hvplot()

:Curve   [timestamp]   (close)

## SMA for the Short and Long Windows Calculation

In [14]:
# Set the variables for short window and long window periods
short_window = 20
middle_window = 50
long_window = 100

In [15]:
# Generate the short and long window simple moving averages (50 and 100 days, respectively)
signals_df["SMA20"] = signals_df["close"].rolling(window=short_window).mean()
signals_df["SMA50"] = signals_df["close"].rolling(window=middle_window).mean()
signals_df["SMA100"] = signals_df["close"].rolling(window=long_window).mean()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,close,SMA20,SMA50,SMA100
timestamp,,,,
2017-06-01 04:00:00+00:00,70.10,NaN,NaN,NaN
2017-06-02 04:00:00+00:00,71.76,NaN,NaN,NaN
2017-06-05 04:00:00+00:00,72.28,NaN,NaN,NaN
2017-06-06 04:00:00+00:00,72.52,NaN,NaN,NaN
2017-06-07 04:00:00+00:00,72.39,NaN,NaN,NaN


,close,SMA20,SMA50,SMA100
timestamp,,,,
2022-05-25 04:00:00+00:00,262.52,267.8745,284.3072,292.1949
2022-05-26 04:00:00+00:00,265.90,266.6880,283.7374,291.5064
2022-05-27 04:00:00+00:00,273.24,266.4740,283.2978,290.9487
2022-05-31 04:00:00+00:00,271.87,265.8440,282.7266,290.5036
2022-06-01 04:00:00+00:00,272.42,265.3760,282.1918,290.0890


## Trading Signals Identification

In [16]:
# Create a column to hold the trading signal
signals_df["Signal"] = 0.0

In [17]:

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df["Signal"][short_window:] = np.where(
    ((signals_df["SMA20"][short_window:] > signals_df["SMA50"][short_window:]) &
    (signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:])), 1.0, 0.0

)

signals_df.tail()


,close,SMA20,SMA50,SMA100,Signal
timestamp,,,,,
2022-05-25 04:00:00+00:00,262.52,267.8745,284.3072,292.1949,0.0
2022-05-26 04:00:00+00:00,265.90,266.6880,283.7374,291.5064,0.0
2022-05-27 04:00:00+00:00,273.24,266.4740,283.2978,290.9487,0.0
2022-05-31 04:00:00+00:00,271.87,265.8440,282.7266,290.5036,0.0
2022-06-01 04:00:00+00:00,272.42,265.3760,282.1918,290.0890,0.0


## Finding the Crossover Points

In [18]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
#signals_df.loc["2015-02-09":"2015-02-17"]

In [19]:
# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(title="Selling Points",
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
exit

:Scatter   [timestamp]   (close)

In [20]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(title="Entry Points",
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
entry

:Scatter   [timestamp]   (close)

In [21]:
# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot( title="Closing Data",
    line_color="black",
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
security_close

:Curve   [timestamp]   (close)

In [22]:
# Visualize moving averages
moving_avgs = signals_df[["SMA20","SMA50", "SMA100"]].hvplot( title = "Moving Averages",
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [23]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="MSFT - SMA20, SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [24]:
# Initialize trade_type column for buys and sells
msft_df['trade_type'] = np.nan

# Initialize variable to hold the previous_price
previous_price = 0

In [25]:
# Initialize a cost/proceeds column for recording trade metrics
msft_df["cost/proceeds"] = np.nan

In [26]:
# Initialize share_size equals 100 and accumulated_shares equals 0
share_size = 10
accumulated_shares = 0

In [27]:
# Loop through the Pandas DataFrame and code the conditions of the trading strategy 
for index, row in msft_df.iterrows():
    
    # buy if the previous price is 0, in other words, buy on the first day
    # set the cost/proceeds column equal to the negative value of the row close price
    # multiplied by the share_size
    if previous_price == 0:
        msft_df.loc[index, "trade_type"] = "buy"
        msft_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)
        accumulated_shares += share_size

    # buy if the current day price is less than the previous day price
    # set the cost/proceeds column equal to the negative value of the row close price
    # multiplied by the share_size
    elif row["close"] < previous_price:
        msft_df.loc[index, "trade_type"] = "buy"
        msft_df.loc[index, "cost/proceeds"] = -(row["close"] * share_size)
        accumulated_shares += share_size
    
    # sell if the current day price is greater than the previous day price
    elif row["close"] > previous_price:
        msft_df.loc[index, "trade_type"] = "sell"

    # else hold if the current day price is equal to the previous day price
    else:
        msft_df.loc[index, "trade_type"] = "hold"
    
    # set the previous_price variable to the close price of the current row
    previous_price = row["close"]
    
    # if the index is the last index of the Dataframe, sell
    # set the cost/proceeds column equal to the row close price multiplied 
    # by the accumulated_shares
    if index == msft_df.index[-1]:
        msft_df.loc[index, "trade_type"] = "sell"
        msft_df.loc[index, "cost/proceeds"] = row["close"] * accumulated_shares

In [28]:
# Run the algorithm and review the DataFrame
msft_df_returns = msft_df

In [29]:
msft_df_returns

,open,high,low,close,volume,symbol,trade_type,cost/proceeds
timestamp,,,,,,,,
2017-06-01 04:00:00+00:00,70.24,70.61,69.4510,70.10,21922908,MSFT,buy,-701.0
2017-06-02 04:00:00+00:00,70.44,71.86,70.2400,71.76,35470261,MSFT,sell,NaN
2017-06-05 04:00:00+00:00,71.97,72.89,71.8100,72.28,33317996,MSFT,sell,NaN
2017-06-06 04:00:00+00:00,72.30,72.62,72.2700,72.52,31511065,MSFT,sell,NaN
2017-06-07 04:00:00+00:00,72.67,72.77,71.9500,72.39,22302769,MSFT,buy,-723.9
...,...,...,...,...,...,...,...,...
2022-05-25 04:00:00+00:00,258.36,264.58,257.1250,262.52,28402776,MSFT,sell,NaN
2022-05-26 04:00:00+00:00,262.29,267.11,261.4294,265.90,24933202,MSFT,sell,NaN
2022-05-27 04:00:00+00:00,268.51,273.34,267.5600,273.24,26809462,MSFT,sell,NaN


In [30]:
weekly_returns = msft_df_returns.groupby(
    by=[
        msft_df_returns.index.isocalendar().year,
        msft_df_returns.index.isocalendar().week
    ]
).sum()
weekly_returns.tail()

open     high        low    close     volume  cost/proceeds
year week                                                                
2022 18    1404.54  1425.53  1378.7100  1408.31  174710799        -8338.6
     19    1322.27  1340.39  1293.0600  1311.10  220565911        -7804.8
     20    1299.84  1313.96  1269.3600  1288.14  161613406        -7597.8
     21    1302.82  1327.86  1293.0444  1321.93  142081588        -2596.2
     22     547.50   552.46   538.9700   544.29   62799550      1522833.3

In [31]:
# Calculate the total profit/loss for 100 share size orders
total_profit_loss = round(msft_df["cost/proceeds"].sum(), 2)

# Print the profit/loss metrics
print(f"The total profit/loss of the trading strategy is ${total_profit_loss}.")

The total profit/loss of the trading strategy is $541013.75.


In [32]:
# Initialize the variable to hold the value of the invested capital
invested_capital = 10000

In [33]:
# Calculate the invested capital by adding the cost of all buy trades
for index, row in msft_df.iterrows():
    if row["trade_type"] == "buy":
        invested_capital = invested_capital + row["cost/proceeds"]

In [34]:
# Calculate the return on investment (ROI)
roi = round((total_profit_loss / -(invested_capital)) * 100, 2)
         
# Print the ROI
print(f"The trading algorithm resulted in a return on investment of {roi}%")

The trading algorithm resulted in a return on investment of 55.51%


In [35]:
# Set initial capital
initial_capital = float(10000)

# Set the share size
share_size = 143

In [36]:
signals_df['Position'] = share_size * signals_df['Signal']

In [37]:
# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [38]:

# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum() 
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
# Print the DataFrame
signals_df.to_csv('../Project_2/DomMSFTcumret.csv', index=True)


OSError: Cannot save file into a non-existent directory: '../Project_2'

In [ ]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="MSFT Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

In [ ]:
# Read the trading_signals.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
signals_df = pd.read_csv(
    Path("DomMSFTcumret.csv"),
    index_col="timestamp",
    parse_dates=True,
    infer_datetime_format=True)

# Review the DataFrame
signals_df.head(10)

In [39]:
# Create a list for the column name
columns = ["Backtest"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio",
    "Sortino Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [40]:
# Calculate annualized return
portfolio_evaluation_df.loc["Annualized Return"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.254134
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [41]:
# Calculate cumulative return
portfolio_evaluation_df.loc["Cumulative Returns"] = signals_df["Portfolio Cumulative Returns"][-1]

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.254134
Cumulative Returns,1.936649
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [42]:
# Calculate annual volatility
portfolio_evaluation_df.loc["Annual Volatility"] = (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.254134
Cumulative Returns,1.936649
Annual Volatility,0.277118
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [43]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc["Sharpe Ratio"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252) / (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.254134
Cumulative Returns,1.936649
Annual Volatility,0.277118
Sharpe Ratio,0.917063
Sortino Ratio,NaN


In [44]:
# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[["Portfolio Daily Returns"]].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,"Downside Returns"] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df["Portfolio Daily Returns"] < 0,
                     "Downside Returns"] = sortino_ratio_df["Portfolio Daily Returns"]**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
timestamp,,
2022-05-25 04:00:00+00:00,0.0,0.0
2022-05-26 04:00:00+00:00,0.0,0.0
2022-05-27 04:00:00+00:00,0.0,0.0
2022-05-31 04:00:00+00:00,0.0,0.0
2022-06-01 04:00:00+00:00,0.0,0.0


In [45]:
# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df["Portfolio Daily Returns"].mean() * 252
)

# Print the result
print(f"Annualized Return: {annualized_return}")

Annualized Return: 0.2541341911709384


In [46]:
# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df["Downside Returns"].mean()) * np.sqrt(252)
)

# Print the result
print(f"Annualized Downside STD: {downside_standard_deviation}")

Annualized Downside STD: 0.18879441196157432


In [47]:
# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc["Sortino Ratio"] = sortino_ratio

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,0.254134
Cumulative Returns,1.936649
Annual Volatility,0.277118
Sharpe Ratio,0.917063
Sortino Ratio,1.34609


In [48]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        "Stock",
        "Entry Date",
        "Exit Date",
        "Shares",
        "Entry Share Price",
        "Exit Share Price",
        "Entry Portfolio Holding",
        "Exit Portfolio Holding",
        "Profit/Loss"]
)

In [49]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row["Entry/Exit"] == 1:
        entry_date = index
        entry_portfolio_holding = row["Portfolio Holdings"]
        share_size = row["Entry/Exit Position"]
        entry_share_price = row["close"]

    elif row["Entry/Exit"] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["close"] * row["Entry/Exit Position"])
        exit_share_price = row["close"]
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                "Stock": "MSFT",
                "Entry Date": entry_date,
                "Exit Date": exit_date,
                "Shares": share_size,
                "Entry Share Price": entry_share_price,
                "Exit Share Price": exit_share_price,
                "Entry Portfolio Holding": entry_portfolio_holding,
                "Exit Portfolio Holding": exit_portfolio_holding,
                "Profit/Loss": profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

/var/folders/jl/7bykfdr560q50k3f7sc9xpfc0000gn/T/ipykernel_46599/2110665004.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
/var/folders/jl/7bykfdr560q50k3f7sc9xpfc0000gn/T/ipykernel_46599/2110665004.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
/var/folders/jl/7bykfdr560q50k3f7sc9xpfc0000gn/T/ipykernel_46599/2110665004.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trade_evaluation_df = trade_evaluation_df.append(
/var/folders/jl/7bykfdr560q50k3f7sc9xpfc0000gn/T/ipykernel_46599/2110665004.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,MSFT,2017-10-20 04:00:00+00:00,2018-04-06 04:00:00+00:00,143.0,78.81,90.23,11269.83,12902.89,1633.06
1,MSFT,2018-04-26 04:00:00+00:00,2018-10-24 04:00:00+00:00,143.0,94.26,102.32,13479.18,14631.76,1152.58
2,MSFT,2019-03-12 04:00:00+00:00,2019-08-23 04:00:00+00:00,143.0,113.62,133.39,16247.66,19074.77,2827.11
3,MSFT,2019-09-23 04:00:00+00:00,2020-03-13 04:00:00+00:00,143.0,139.14,158.83,19897.02,22712.69,2815.67
4,MSFT,2020-04-22 04:00:00+00:00,2020-04-24 04:00:00+00:00,143.0,173.52,174.55,24813.36,24960.65,147.29
5,MSFT,2020-05-27 04:00:00+00:00,2020-09-28 04:00:00+00:00,143.0,181.81,209.44,25998.83,29949.92,3951.09
6,MSFT,2020-10-21 04:00:00+00:00,2020-11-16 05:00:00+00:00,143.0,214.8,217.23,30716.4,31063.89,347.49
7,MSFT,2020-11-17 05:00:00+00:00,2020-11-18 05:00:00+00:00,143.0,214.46,211.08,30667.78,30184.44,-483.34
8,MSFT,2020-11-20 05:00:00+00:00,2020-12-14 05:00:00+00:00,143.0,210.39,214.2,30085.77,30630.6,544.83
9,MSFT,2020-12-18 05:00:00+00:00,2021-03-23 04:00:00+00:00,143.0,218.59,237.58,31258.37,33973.94,2715.57


# Creating Functions for Alpaca .Py File